In [207]:
import numpy as np
import pickle
import time
import lime
import lime.lime_tabular
import deeplift
import tensorflow as tf
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Conv1D, MaxPooling1D, Flatten, Dropout, Activation, Layer, Input,embeddings
from collections import Counter
from copy import deepcopy

from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation
from keras import backend as K
from deeplift.conversion import kerasapi_conversion as kc
from tensorflow import keras

In [208]:
class LeeOscillator:
    def __init__(self, a=[1, 1, 1, 1, -1, -1, -1, -1], b=[0.6, 0.6, -0.5, 0.5, -0.6, -0.6, -0.5, 0.5], K=50, N=10):
        self.a = a
        self.b = b
        self.K = K
        self.N = N

    def _initialize_tensors(self, x, N):
        shape = tf.shape(x)
        rank = tf.rank(x)

        if rank == 2:
            x = tf.expand_dims(tf.expand_dims(x, axis=1), axis=1)
            shape = tf.shape(x)
        elif rank == 4 and shape[1] > 1:
            x = tf.transpose(x, [0, 2, 3, 1])
            shape = tf.shape(x)

        init_shape = tf.concat([[N], shape], axis=0)
        u = tf.zeros(init_shape, dtype=tf.float32)
        v = tf.zeros(init_shape, dtype=tf.float32)
        z = tf.zeros(init_shape, dtype=tf.float32)
        u = tf.tensor_scatter_nd_update(u, [[0]], [u[0] + 0.2])
        z = tf.tensor_scatter_nd_update(z, [[0]], [z[0] + 0.2])

        return u, v, z, x

    def Tanh(self, x):
        N = np.random.randint(1, self.N + 1)
        u, v, z, x = self._initialize_tensors(x, N)

        for t in range(N - 1):
            u = tf.tensor_scatter_nd_update(u, [[t + 1]], [
                tf.math.tanh(self.a[0] * u[t] - self.a[1] * v[t] + self.a[2] * z[t] + self.a[3] * x)])
            v = tf.tensor_scatter_nd_update(v, [[t + 1]], [
                tf.math.tanh(self.a[6] * z[t] - self.a[4] * u[t] - self.a[5] * v[t] + self.a[7] * x)])
            w = tf.math.tanh(x)
            z = tf.tensor_scatter_nd_update(z, [[t + 1]], [(v[t + 1] - u[t + 1]) * tf.math.exp(-self.K * tf.math.pow(x, 2)) + w])

        if tf.rank(x) == 4 and tf.shape(x)[1] > 1:
            z = tf.transpose(z[-1], [0, 3, 1, 2])

        return z[-1]

    def Sigmoid(self, x):
        N = np.random.randint(1, self.N + 1)
        u, v, z, x = self._initialize_tensors(x, N)

        for t in range(N - 1):
            u = tf.tensor_scatter_nd_update(u, [[t + 1]], [
                tf.math.sigmoid(self.b[0] * u[t] - self.b[1] * v[t] + self.b[2] * z[t] + self.b[3] * x)])
            v = tf.tensor_scatter_nd_update(v, [[t + 1]], [
                tf.math.sigmoid(self.b[6] * z[t] - self.b[4] * u[t] - self.b[5] * v[t] + self.b[7] * x)])
            w = tf.math.sigmoid(x)
            z = tf.tensor_scatter_nd_update(z, [[t + 1]], [(v[t + 1] - u[t + 1]) * tf.math.exp(-self.K * tf.math.pow(x, 2)) + w])

        if tf.rank(x) == 4 and tf.shape(x)[1] > 1:
            z = tf.transpose(z[-1], [0, 3, 1, 2])

        return z[-1]

    def ReLU(self, x):
        N = np.random.randint(1, self.N + 1)
        u, v, z, x = self._initialize_tensors(x, N)

        for t in range(N - 1):
            u = tf.tensor_scatter_nd_update(u, [[t + 1]], [
                tf.nn.relu(self.a[0] * u[t] - self.a[1] * v[t] + self.a[2] * z[t] + self.a[3] * x)])
            v = tf.tensor_scatter_nd_update(v, [[t + 1]], [
                tf.nn.relu(self.a[6] * z[t] - self.a[4] * u[t] - self.a[5] * v[t] + self.a[7] * x)])
            w = tf.nn.relu(x)
            z = tf.tensor_scatter_nd_update(z, [[t + 1]], [(v[t + 1] - u[t + 1]) * tf.math.exp(-self.K * tf.math.pow(x, 2)) + w])

        if tf.rank(x) == 4 and tf.shape(x)[1] > 1:
            z = tf.transpose(z[-1], [0, 3, 1, 2])

        return z[-1]

    def LeakyReLU(self, x, alpha=0.1):
        N = np.random.randint(1, self.N + 1)
        u, v, z, x = self._initialize_tensors(x, N)

        for t in range(N - 1):
            u = tf.tensor_scatter_nd_update(u, [[t + 1]], [
                tf.nn.leaky_relu(self.a[0] * u[t] - self.a[1] * v[t] + self.a[2] * z[t] + self.a[3] * x, alpha=alpha)])
            v = tf.tensor_scatter_nd_update(v, [[t + 1]], [
                tf.nn.leaky_relu(self.a[6] * z[t] - self.a[4] * u[t] - self.a[5] * v[t] + self.a[7] * x, alpha=alpha)])
            w = tf.nn.leaky_relu(x, alpha=alpha)
            z = tf.tensor_scatter_nd_update(z, [[t + 1]], [(v[t + 1] - u[t + 1]) * tf.math.exp(-self.K * tf.math.pow(x, 2)) + w])

        if tf.rank(x) == 4 and tf.shape(x)[1] > 1:
            z = tf.transpose(z[-1], [0, 3, 1, 2])

        return z[-1]

In [209]:
# 自定义激活层
class LeeOscillatorTanhLayer(Layer):
    def __init__(self, **kwargs):
        super(LeeOscillatorTanhLayer, self).__init__(**kwargs)
        self.oscillator = LeeOscillator()

    def call(self, inputs):
        return self.oscillator.Tanh(inputs)

class LeeOscillatorSigmoidLayer(Layer):
    def __init__(self, **kwargs):
        super(LeeOscillatorSigmoidLayer, self).__init__(**kwargs)
        self.oscillator = LeeOscillator()

    def call(self, inputs):
        return self.oscillator.Sigmoid(inputs)

class LeeOscillatorReLULayer(Layer):
    def __init__(self, **kwargs):
        super(LeeOscillatorReLULayer, self).__init__(**kwargs)
        self.oscillator = LeeOscillator()

    def call(self, inputs):
        return self.oscillator.ReLU(inputs)

class LeeOscillatorLeakyReLULayer(Layer):
    def __init__(self, **kwargs):
        super(LeeOscillatorLeakyReLULayer, self).__init__(**kwargs)
        self.oscillator = LeeOscillator()

    def call(self, inputs):
        return self.oscillator.LeakyReLU(inputs)

In [210]:
custom_objects = {'LeeOscillatorReLULayer': LeeOscillatorReLULayer}
model = load_model("NN.h5", custom_objects=custom_objects)

new_true_model = Sequential()
for layer in model.layers:
    if isinstance(layer, keras.layers.Embedding):
        # 替换Embedding层
        new_true_model.add(Dense(units=layer.output_dim, activation='relu'))
    else:
        new_true_model.add(layer)

# 保存修改后的模型
new_true_model.save("modified_NN.h5")

In [211]:
# Load Data
X_train = np.load("X_train.npy")
X_test = np.load("X_test.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")

oh_y_train = np.load("oh_y_train.npy")
oh_y_test = np.load("oh_y_test.npy")

In [212]:
# 修改此处1：只读取前1000个数据样本
X_train = X_train[:1000]
X_test = X_test[:1000]
y_train = y_train[:1000]
y_test = y_test[:1000]
oh_y_train = oh_y_train[:1000]
oh_y_test = oh_y_test[:1000]

In [213]:
# For indexing
nn_preds_test = model.predict_classes(X_test)
nn_preds_train = model.predict_classes(X_train)

In [214]:


# Manually construct a new fake model without the Embedding layer
inputs = Input(shape=(500, 128))  # Input shape should match the output shape of the Embedding layer
x = Conv1D(32, 5, padding='same', name='conv1')(inputs)
x = Activation("relu", name='activation1')(x)
x = MaxPooling1D(pool_size=2, name='maxpool1')(x)
x = Conv1D(64, 3, padding='same', name='conv2')(x)
x = Activation("relu", name='activation2')(x)
x = MaxPooling1D(pool_size=2, name='maxpool2')(x)
x = Flatten(name='flatten')(x)
x = Dense(128, name='dense1')(x)
x = Activation("relu", name='activation3')(x)
x = Dropout(0.2, name='dropout1')(x)
x = Dense(50, name='dense2')(x)
x = Activation("relu", name='activation4')(x)
x = Dropout(0.2, name='dropout2')(x)
outputs = Dense(46, name='output')(x)
outputs = Activation("softmax", name='activation5')(outputs)

# Create a new model without the Embedding layer
fake_model = Model(inputs, outputs)

# Transfer the weights from the trained model to the new model
fake_model.get_layer(name='conv1').set_weights(model.get_layer(name='conv1d_1').get_weights())
fake_model.get_layer(name='conv2').set_weights(model.get_layer(name='conv1d_2').get_weights())
fake_model.get_layer(name='dense1').set_weights(model.get_layer(name='dense_1').get_weights())
fake_model.get_layer(name='dense2').set_weights(model.get_layer(name='dense_2').get_weights())
fake_model.get_layer(name='output').set_weights(model.get_layer(name='dense_3').get_weights())

# Save the new model to file
fake_model.save('fake_model_NN_no_embedding.h5')

In [215]:
# Load the model to access the weights
model = load_model('fake_model_NN_no_embedding.h5')

# Print the names of the layers in the trained model
for layer in model.layers:
    print(layer.name)


input_6
conv1
activation1
maxpool1
conv2
activation2
maxpool2
flatten
dense1
activation3
dropout1
dense2
activation4
dropout2
output
activation5


c:\Users\Bill\.conda\envs\python3.6\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [216]:
from copy import deepcopy
from deeplift.conversion import kerasapi_conversion as kc

# Assuming you have pre-computed embeddings for X_train and X_test
# For simplicity, here we generate random embeddings. Replace this with actual embeddings.
precomputed_embeddings_train = np.random.rand(X_train.shape[0], 500, 128)
precomputed_embeddings_test = np.random.rand(X_test.shape[0], 500, 128)

In [217]:

# Convert the new model using DeepLIFT
deeplift_model = deeplift.conversion.kerasapi_conversion.convert_model_from_saved_files(
    "fake_model_NN_no_embedding.h5",
    nonlinear_mxts_mode=deeplift.layers.NonlinearMxtsMode.DeepLIFT_GenomicsDefault)

# Define the layer to propagate contribution scores
find_scores_layer_name = 'dense1_0'
pre_activation_target_layer_name = 'output_0'

# Get the target contribution function
deeplift_contribs_func = deeplift_model.get_target_contribs_func(
    find_scores_layer_name=find_scores_layer_name,
    pre_activation_target_layer_name=pre_activation_target_layer_name)

# Iterate scores for each output neuron one at a time
for i in range(model.get_weights()[-1].shape[0]):
    train = np.array(deeplift_contribs_func(task_idx=i,
                                            input_data_list=[precomputed_embeddings_train],
                                            batch_size=10,
                                            progress_update=1000))

    test = np.array(deeplift_contribs_func(task_idx=i,
                                           input_data_list=[precomputed_embeddings_test],
                                           batch_size=10,
                                           progress_update=1000))

    train = np.array(train)
    test = np.array(test)
    train = np.expand_dims(train, 1)
    test = np.expand_dims(test, 1)

    if i == 0:
        X_train_deeplift = deepcopy(train)
        X_test_deeplift = deepcopy(test)
    else:
        X_train_deeplift = np.append(X_train_deeplift, train, axis=1)
        X_test_deeplift = np.append(X_test_deeplift, test, axis=1)

c:\Users\Bill\.conda\envs\python3.6\lib\site-packages\deeplift\conversion\kerasapi_conversion.py:359: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  str_data = h5py.File(h5_file).attrs["model_config"]
c:\Users\Bill\.conda\envs\python3.6\lib\site-packages\deeplift\conversion\kerasapi_conversion.py:366: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  model_weights = h5py.File(h5_file)


nonlinear_mxts_mode is set to: DeepLIFT_GenomicsDefault
For layer activation1_0 the preceding linear layer is conv1_0 of type Conv1D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
Heads-up: current implementation assumes maxpool layer is followed by a linear transformation (conv/dense layer)
For layer activation2_0 the preceding linear layer is conv2_0 of type Conv1D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
Heads-up: current implementation assumes maxpool layer is followed by a linear transformation (conv/dense layer)
For layer activation3_0 the preceding linear layer is dense1_0 of type Dense;
In accordance with nonlinear_mxts_modeDeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to RevealCancel
For layer activation4_0 the preceding linear layer is dense2_0 of type Dense;
In accordance with nonlinear_mxts_modeDeepLIFT_GenomicsDefault we 

In [218]:
train = deepcopy(X_train_deeplift)
test = deepcopy(X_test_deeplift)

In [219]:
X_train_deeplift = list()
X_test_deeplift = list()

for i in range(len(nn_preds_train)):
    index = nn_preds_train[i]
    X_train_deeplift.append(train[i][index])

for i in range(len(nn_preds_test)):
    index = nn_preds_test[i]
    X_test_deeplift.append(test[i][index])

In [220]:
X_train_deeplift = np.array(X_train_deeplift)
X_test_deeplift = np.array(X_test_deeplift)

In [221]:
print("Training:", X_train_deeplift.shape)
print("Testing:", X_test_deeplift.shape)

np.save("new_X_train_deeplift", X_train_deeplift)
np.save("new_X_test_deeplift", X_test_deeplift)

Training: (1000, 128)
Testing: (1000, 128)


In [222]:
#### NB: MAKE SURE CORRECT MODEL
custom_objects = {'LeeOscillatorReLULayer': LeeOscillatorReLULayer}
model = load_model("modified_NN.h5", custom_objects=custom_objects)

In [223]:
def get_activations(model, model_inputs, print_shape_only=False, layer_name=None):
    activations = []
    inp = model.input

    model_multi_inputs_cond = True
    if not isinstance(inp, list):
        inp = [inp]
        model_multi_inputs_cond = False

    outputs = [layer.output for layer in model.layers if
               layer.name == layer_name or layer_name is None]

    funcs = [K.function(inp + [K.learning_phase()], [out]) for out in outputs]

    if model_multi_inputs_cond:
        list_inputs = []
        list_inputs.extend(model_inputs)
        list_inputs.append(0.)
    else:
        list_inputs = [model_inputs, 0.]

    layer_outputs = [func(list_inputs)[0] for func in funcs]
    for layer_activations in layer_outputs:
        activations.append(layer_activations)
        if print_shape_only:
            print(layer_activations.shape)
        else:
            print(layer_activations)

    return activations

X_train_act = list()
X_test_act = list()

In [224]:
# 修改此处，根据修改1处的数量修改for i in range(0, x, 1000): 里的x
for i in range(0, 1000, 1000):
    print(i)
    start = i
    end = i + 1000

    X_train_act_seg = get_activations(model, X_train[start: end],print_shape_only=True)[-8]
    X_train_act.append(X_train_act_seg)

# 修改此处，根据修改1处的数量修改for i in range(0, x, 1000): 里的x
for i in range(0, 1000, 1000):
    print(i)
    start = i
    end = i + 1000

    X_test_act_seg = get_activations(model, X_test[start: end],print_shape_only=True)[-8]
    X_test_act.append(X_test_act_seg)

X_train_act_new = list()
X_test_act_new = list()

for group in X_train_act:
    for i in range(len(group)):
        X_train_act_new.append(group[i])

for group in X_test_act:
    for i in range(len(group)):
        X_test_act_new.append(group[i])

X_train_act = np.array(X_train_act_new)
X_test_act = np.array(X_test_act_new)

np.save("X_train_act", X_train_act)
np.save("X_test_act", X_test_act)

0
(1000, 500, 128)
(1000, 500, 128)
(1000, 500, 32)
(1000, 500, 32)
(1000, 250, 32)
(1000, 250, 64)
(1000, 250, 64)
(1000, 125, 64)
(1000, 8000)
(1000, 128)
(1000, 128)
(1000, 128)
(1000, 50)
(1000, 50)
(1000, 50)
(1000, 46)
(1000, 46)
0
(1000, 500, 128)
(1000, 500, 128)
(1000, 500, 32)
(1000, 500, 32)
(1000, 250, 32)
(1000, 250, 64)
(1000, 250, 64)
(1000, 125, 64)
(1000, 8000)
(1000, 128)
(1000, 128)
(1000, 128)
(1000, 50)
(1000, 50)
(1000, 50)
(1000, 46)
(1000, 46)


In [225]:
import pandas as pd
import numpy as np
import pickle
import time

from collections import Counter

from copy import deepcopy
import numpy as np
import pickle
import time
import lime
import lime.lime_tabular
import deeplift
import tensorflow as tf
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Conv1D, MaxPooling1D, Flatten, Dropout, Activation, Layer, Input,embeddings
from collections import Counter
from copy import deepcopy

from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation
from keras import backend as K
from deeplift.conversion import kerasapi_conversion as kc
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier

# For keras dependencise
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras import backend as K

# For LRP Visuals
from deepexplain.tensorflow import DeepExplain
from keras.models import Model

import matplotlib.pyplot as plt

In [226]:
# Pick a value of k for the dataset
k = 1

In [227]:
custom_objects = {'LeeOscillatorReLULayer': LeeOscillatorReLULayer}
model = load_model("modified_NN.h5", custom_objects=custom_objects)

In [228]:
# Load Data
X_train = np.load("X_train.npy")
X_test = np.load("X_test.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")

# 修改此处1：只读取前1000个数据样本
X_train = X_train[:1000]
X_test = X_test[:1000]
y_train = y_train[:1000]
y_test = y_test[:1000]


# Load the feature activations
X_train_act = np.load("X_train_act.npy")
X_test_act = np.load("X_test_act.npy")
# 将三维数据重塑为二维
X_train_act = X_train_act.reshape(X_train_act.shape[0], -1)
X_test_act = X_test_act.reshape(X_test_act.shape[0], -1)


# Load DeepLIFT contributions
X_train_cont = np.load("new_X_train_deeplift.npy")
X_test_cont = np.load("new_X_test_deeplift.npy")

In [229]:
nn_pred = model.predict_classes(X_test)
# 确定数据形状
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (1000, 500)
X_test shape: (1000, 500)


In [230]:
# 重新定义数据形状以适应KNN分类器
X_train_knn = X_train.reshape(X_train.shape[0], 50*10)
X_test_knn = X_test.reshape(X_test.shape[0], 50*10)
print("X_train_knn shape:", X_train_knn.shape)
print("X_test_knn shape:", X_test_knn.shape)

X_train_knn shape: (1000, 500)
X_test_knn shape: (1000, 500)


In [231]:

print("C-DeepLIFT-KNN:", X_train_cont.shape)
print("Activations:", X_train_act.shape)
print("Training:", X_train.shape)
print("Training k-NN:", X_train_knn.shape)
print("y_test:", y_test.shape)


C-DeepLIFT-KNN: (1000, 128)
Activations: (1000, 128)
Training: (1000, 500)
Training k-NN: (1000, 500)
y_test: (1000,)


In [232]:
#X_train_cont = np.sum(X_train_cont, axis=2)
#print("C-DeepLIFT-KNN:", X_train_cont.shape)
#X_test_cont = np.sum(X_test_cont, axis=2)
#print("C-DeepLIFT-KNN:", X_test_cont.shape)

In [233]:
techniques = [
    ["k-NN*", X_train_act, X_test_act],
    ["C-DeepLIFT", X_train_cont, X_test_cont]
]


for item in techniques:
    technique = item[0]
    train = item[1]
    test = item[2]

    print("==================================================")
    print(technique)
    print("==================================================")

    kNN = KNeighborsClassifier(n_neighbors=9, algorithm="brute")
    kNN.fit(train, y_train)

    knn_predictions_test = kNN.predict(test)
    print("Accuracy:", accuracy_score(y_test, knn_predictions_test))
    print(confusion_matrix(y_test, knn_predictions_test, labels=None, sample_weight=None))
    print(" ")

    # What's the % that's different?
    correct = 0
    for i in range(len(nn_pred)):
        if knn_predictions_test[i] == nn_pred[i]:
            correct += 1
    print("Agreement", correct / len(nn_pred))

k-NN*
Accuracy: 0.627
[[ 0  0  0 ...  0  0  0]
 [ 0 17  1 ...  0  0  0]
 [ 0  1  1 ...  0  0  0]
 ...
 [ 1  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
 
Agreement 0.741
C-DeepLIFT
Accuracy: 0.636
[[ 0  0  0 ...  0  0  0]
 [ 0 28  0 ...  0  0  0]
 [ 0  4  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 1  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
 
Agreement 0.857


In [234]:
techniques = [
    ["k-NN*", X_train_act, X_test_act],
    ["C-DeepLIFT", X_train_cont, X_test_cont]
]


for item in techniques:
    technique = item[0]
    train = item[1]
    test = item[2]

    print("==================================================")
    print(technique)
    print("==================================================")

    kNN = KNeighborsClassifier(n_neighbors=9, algorithm="brute")
    kNN.fit(train, y_train)

    knn_predictions_test = kNN.predict(test)
    print("Accuracy:", accuracy_score(y_test, knn_predictions_test))
    print(confusion_matrix(y_test, knn_predictions_test, labels=None, sample_weight=None))
    print(" ")

    # What's the % that's different?
    correct = 0
    for i in range(len(nn_pred)):
        if knn_predictions_test[i] == nn_pred[i]:
            correct += 1
    print("Agreement", correct / len(nn_pred))

k-NN*
Accuracy: 0.627
[[ 0  0  0 ...  0  0  0]
 [ 0 17  1 ...  0  0  0]
 [ 0  1  1 ...  0  0  0]
 ...
 [ 1  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
 
Agreement 0.741
C-DeepLIFT
Accuracy: 0.636
[[ 0  0  0 ...  0  0  0]
 [ 0 28  0 ...  0  0  0]
 [ 0  4  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 1  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
 
Agreement 0.857


In [235]:
del_idx = list()
for i in range(len(X_train)):
    if y_train[i] != model.predict_classes(np.array([X_train[i]]))[0]:
        del_idx.append(i)

In [236]:
# Modify shape for next loop
y_train = np.array([y_train]).T

In [237]:
to_modify = [X_train, y_train, X_train_act,
             X_train_cont]

modified = list()
for i in range(len(to_modify)):
    matrix = deepcopy(to_modify[i].tolist())
    for index in sorted(del_idx, reverse=True):
        del matrix[index]
    modified.append(matrix)

In [238]:
names = ['X_train_del', 'y_train_del', 'X_train_act_del',
         'X_train_cont_del']

for i in range(len(names)):
    matrix = np.array(modified[i])
    name = names[i]
    # for y_train
    if i == 1:
        matrix = matrix.T[0]
    np.save(name, matrix)

In [239]:
X_train = np.load("X_train_del.npy")
X_test = np.load("X_test.npy")
y_train = np.load("y_train_del.npy")
y_test = np.load("y_test.npy")

# 修改此处1：只读取前1000个数据样本
y_test = y_test[:1000]

X_train_act = np.load("X_train_act_del.npy")
X_train_cont = np.load("X_train_cont_del.npy")

In [240]:
nn_pred = model.predict_classes(X_test)
# 修改此处1：只读取前1000个数据样本
nn_pred = nn_pred[:1000]

In [241]:
print("C-DeepLIFT-KNN:", X_train_cont.shape)
print("Activations:", X_train_act.shape)
print("Training:", X_train.shape)
print("Training k-NN:", X_train_knn.shape)
print("y_test:", y_test.shape)

C-DeepLIFT-KNN: (698, 128)
Activations: (698, 128)
Training: (698, 500)
Training k-NN: (1000, 500)
y_test: (1000,)


In [242]:
techniques = [
    ["k-NN*", X_train_act, X_test_act],
    ["C-DeepLIFT", X_train_cont, X_test_cont]
]

for item in techniques:
    technique = item[0]
    train = item[1]
    test = item[2]

    print("==================================================")
    print(technique)
    print("==================================================")

    kNN = KNeighborsClassifier(n_neighbors=9, algorithm="brute")
    kNN.fit(train, y_train)

    knn_predictions_test = kNN.predict(test)
    print("Accuracy:", accuracy_score(y_test, knn_predictions_test))
    print(confusion_matrix(y_test, knn_predictions_test, labels=None, sample_weight=None))
    print(" ")

    # What's the % that's different?
    correct = 0
    for i in range(len(nn_pred)):
        if knn_predictions_test[i] == nn_pred[i]:
            correct += 1
    print("Agreement", correct / len(nn_pred))

k-NN*
Accuracy: 0.63
[[ 0  0  0 ...  0  0  0]
 [ 0 18  0 ...  0  0  0]
 [ 0  3  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  1  0 ...  0  0  0]]
 
Agreement 0.84
C-DeepLIFT
Accuracy: 0.644
[[ 0  0  0 ...  0  0  0]
 [ 0 25  0 ...  0  0  0]
 [ 0  2  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
 
Agreement 0.904
